In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer, LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
import ast

In [2]:
# Cargar solo los datos necesarios
movies = pd.read_csv(r'Datasets/transformed_movies.csv')
movies_reduced = movies[['title', 'popularity', 'collection', 'genres_encoded']].copy()

# Rellenar valores nulos en la columna de colecciones
movies_reduced['collection'] = movies_reduced['collection'].fillna('No Collection')

# Asegurarse de que los géneros estén en formato de lista
def process_genres(genre_str):
    try:
        return ast.literal_eval(genre_str)
    except ValueError:
        return []

movies_reduced['genres_encoded'] = movies_reduced['genres_encoded'].apply(process_genres)

# Procesar géneros usando MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genres_matrix = mlb.fit_transform(movies_reduced['genres_encoded'])
movies_features = pd.DataFrame(genres_matrix, columns=mlb.classes_)

# Procesar colecciones como una sola variable escalada
collection_counts = movies_reduced['collection'].value_counts()
movies_reduced['collection_weighted'] = movies_reduced['collection'].map(collection_counts).fillna(0)

scaler = StandardScaler()
movies_reduced['collection_weighted_scaled'] = scaler.fit_transform(movies_reduced[['collection_weighted']])

# Combinar características: géneros y colecciones escaladas
movies_features['collection_scaled'] = movies_reduced['collection_weighted_scaled'].values

# Agregar popularidad como una característica más
movies_features['popularity'] = movies_reduced['popularity'].values

# Guardar la matriz combinada en un archivo comprimido
np.savez_compressed("Datasets/movies_features.npz", movies_features.values)

# Guardar el DataFrame reducido como CSV
movies_reduced[['title', 'popularity', 'collection', 'genres_encoded']].to_csv(r'Datasets/movies_reduced.csv', index=False)

